A simple walkthrough of how to code a fully connected neural network
using the PyTorch library. For demonstration we train it on the very
common MNIST dataset of handwritten digits. In this code we go through
how to create the network as well as initialize a loss function, optimizer,
check accuracy and more.

In [1]:
# Importing libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
# Creating a simple fully connected nural network

In [4]:
class ANN(nn.Module):
    def __init__(self, input_size, output_classes):
        super(ANN, self).__init__()
        
        #First fully connected layer
        self.first_fc_layer = nn.Linear(input_size, 50)
        self.second_fc_layer = nn.Linear(50, output_classes)
        
    def forward(self, x):
        #Pass through first layer
        x = F.relu(self.first_fc_layer(x))
        
        #output layer
        x = self.second_fc_layer(x)
        
        return x

In [5]:
# Set devices

In [6]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Hyperparameters

In [8]:
input_shape = 784
output_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 5

In [9]:
# Load Dataset

In [10]:
# Load Data
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [11]:
# Initialized Network

In [12]:
model = ANN(input_size = input_shape, output_classes = output_classes).to(device)

In [13]:
# Loss and Optimizer

In [14]:
criterion = nn.CrossEntropyLoss()

In [15]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
# Train Network

In [17]:
import numpy as np

In [18]:
# lists to store train and test accuracy
print("Starting Training")
print("="*100)

for epoch in range(num_epoch):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # Forward
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()
    print(f"Epoch {epoch} is Completed.")
    print("-"*60)
print("Training Completed")
print("="*100)

Starting Training
Epoch 0 is Completed.
------------------------------------------------------------
Epoch 1 is Completed.
------------------------------------------------------------
Epoch 2 is Completed.
------------------------------------------------------------
Epoch 3 is Completed.
------------------------------------------------------------
Epoch 4 is Completed.
------------------------------------------------------------
Training Completed


In [19]:
# Check Accuracy

In [20]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
        # Loop through the data
        for x, y in loader:

            # Move data to device
            x = x.to(device=device)
            y = y.to(device=device)

            # Get to correct shape
            x = x.reshape(x.shape[0], -1)

            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)

            # Check how many we got correct
            num_correct += (predictions == y).sum()

            # Keep track of number of samples
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

In [21]:
# Check accuracy on training & test to see how good our model
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 97.25
Accuracy on test set: 96.31
